## Agrupación y Segmentación de Vecindarios en la ciudad de Toronto

 _*Descripción de la actividad:*_
 
 
 En esta tarea, se le pedirá que explore, segmente y agrupe los vecindarios de la ciudad de Toronto. Sin embargo, a diferencia de Nueva York, los datos del vecindario no están disponibles en Internet. Lo interesante del campo de la ciencia de datos es que cada proyecto puede ser desafiante en su forma única, por lo que debe aprender a ser ágil y refinar la habilidad para aprender nuevas bibliotecas y herramientas rápidamente según el proyecto.

Para los datos de vecindarios de Toronto, existe una página de Wikipedia que tiene toda la información que necesitamos para explorar y agrupar los vecindarios de Toronto. Se le pedirá que rastree la página de Wikipedia y discuta los datos, los limpie y luego los lea en un marco de datos de pandas para que esté en un formato estructurado como el conjunto de datos de Nueva York.

Una vez que los datos están en un formato estructurado, puede replicar el análisis que hicimos en el conjunto de datos de la ciudad de Nueva York para explorar y agrupar los vecindarios en la ciudad de Toronto.

In [1]:
import pandas as pd
import numpy as np

In [2]:
#Llamamos a la URL de wikipedia para recuperar la información con la que vamos a trabajar
df = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M") #get Canada data From wikipedia
df = df[0]
df = df.melt()['value']
df

0                                        M1ANot assigned
1                        M1BScarborough(Malvern / Rouge)
2      M1CScarborough(Rouge Hill / Port Union / Highl...
3      M1EScarborough(Guildwood / Morningside / West ...
4                                 M1GScarborough(Woburn)
                             ...                        
175    M9VEtobicoke(South Steeles / Silverstone / Hum...
176    M9WEtobicokeNorthwest(Clairville / Humberwood ...
177                                      M9XNot assigned
178                                      M9YNot assigned
179                                      M9ZNot assigned
Name: value, Length: 180, dtype: object

In [3]:
#Recorremos los datos descargados y extraemos los valores de "códigoPostal", "Vecindario" y "Municipio" de los strings 
df_clean = pd.DataFrame()
df_clean['CódigoPostal'] = df.apply(lambda x : x[:3])
df_clean['location_tmp'] = df.apply(lambda x : x[3:])
#Borramos las filas que tienen municipio "no asignado"
df_clean = df_clean[df_clean['location_tmp'] != 'Not assigned']
#df_clean['Vecindario'] = np.where(df['location_tmp'] == 'Not assigned', , )
df_clean['Municipio'] = df_clean['location_tmp'].apply(lambda x: str(x).split('(')[0])
df_clean['Vecindario'] = df_clean['location_tmp'].apply(lambda x: str(x).split('(')[1].replace(')','').replace('/',',' ))
#si el Vecindario está como "No asignado", pero tiene municipio asignado se emplea el valor del Municipio como Vecindario
df_clean['Vecindario'] = np.where(df_clean['Vecindario'] == 'Not assigned', df_clean['Municipio'], df_clean['Vecindario'])
df_clean=df_clean[['CódigoPostal','Municipio', 'Vecindario']].reset_index(drop=True)

In [4]:
df_clean['CódigoPostal'].values

array(['M1B', 'M1C', 'M1E', 'M1G', 'M1H', 'M1J', 'M1K', 'M1L', 'M1M',
       'M1N', 'M1P', 'M1R', 'M1S', 'M1T', 'M1V', 'M1W', 'M1X', 'M2H',
       'M2J', 'M2K', 'M2L', 'M2M', 'M2N', 'M2P', 'M2R', 'M3A', 'M3B',
       'M3C', 'M3H', 'M3J', 'M3K', 'M3L', 'M3M', 'M3N', 'M4A', 'M4B',
       'M4C', 'M4E', 'M4G', 'M4H', 'M4J', 'M4K', 'M4L', 'M4M', 'M4N',
       'M4P', 'M4R', 'M4S', 'M4T', 'M4V', 'M4W', 'M4X', 'M4Y', 'M5A',
       'M5B', 'M5C', 'M5E', 'M5G', 'M5H', 'M5J', 'M5K', 'M5L', 'M5M',
       'M5N', 'M5P', 'M5R', 'M5S', 'M5T', 'M5V', 'M5W', 'M5X', 'M6A',
       'M6B', 'M6C', 'M6E', 'M6G', 'M6H', 'M6J', 'M6K', 'M6L', 'M6M',
       'M6N', 'M6P', 'M6R', 'M6S', 'M7A', 'M7R', 'M7Y', 'M8V', 'M8W',
       'M8X', 'M8Y', 'M8Z', 'M9A', 'M9B', 'M9C', 'M9L', 'M9M', 'M9N',
       'M9P', 'M9R', 'M9V', 'M9W'], dtype=object)

In [5]:
print( 'Nº de filas de la tabla resultante: ' + str(df_clean.shape[0]))

Nº de filas de la tabla resultante: 103


#### Ahora pasaremos a la segunda parte, en la que extraeremos las cordenamas de cada código postal:

In [6]:
#Importamos la librería para recuperar las cordenadas
import geocoder # import geocoder

In [21]:
# loop until you get the coordinates
## NOT WORKING; Method returning <[REQUEST_DENIED] Google - Geocode [empty]>
for i in range(len(df_clean['CódigoPostal'])):
    lat_lng_coords = None
    postal_code = df_clean['CódigoPostal'][i]
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng

    df_clean['Latitude'][i] = lat_lng_coords[0]
    df_clean['Longitude'][i] = lat_lng_coords[1]
    

In [22]:
# loop until you get the coordinates
## NOT WORKING; Method returning <[REQUEST_DENIED] Google - Geocode [empty]>
## USING GIVEN CSV TO LOAD COORDINATES
city_data = pd.read_csv('Geospatial_Coordinates.csv')
city_data = city_data.rename(columns = {'Postal Code' : 'CódigoPostal'})
city_data

,CódigoPostal,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [25]:
df_clean = pd.merge(df_clean,city_data, on='CódigoPostal', how='inner')
df_clean

,CódigoPostal,Municipio,Vecindario,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village , St. Phillips , Martin Grov...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles , Silverstone , Humbergate , Jam...",43.739416,-79.588437


## Parte 3 : Explore y agrupe los vecindarios de toronto

In [26]:
# Primero analizaremos los municipios que contienen la palabra "Toronto"
#siendo ese el resultado:
df_toronto = df_clean[df_clean['Municipio'].str.contains('Toronto')]

In [27]:
df_toronto

,CódigoPostal,Municipio,Vecindario,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
40,M4J,East YorkEast Toronto,The Danforth East,43.685347,-79.338106
41,M4K,East Toronto,"The Danforth West , Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar , The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
47,M4S,Central Toronto,Davisville,43.704324,-79.388790
48,M4T,Central Toronto,"Moore Park , Summerhill East",43.689574,-79.383160


#### Analizaremos el número de vecindarios que agrua cada municipio viendo cuál es el que mayor número de barios alberga y el que menos

In [28]:
grouped_toronto = df_toronto.groupby("Municipio").size()

In [29]:
grouped_toronto.sort_values()

Municipio
Downtown TorontoStn A PO Boxes25 The Esplanade                   1
East TorontoBusiness reply mail Processing Centre969 Eastern     1
East YorkEast Toronto                                            1
East Toronto                                                     4
West Toronto                                                     6
Central Toronto                                                  9
Downtown Toronto                                                17
dtype: int64

> se puede ver que el municipio que mayor número de vecindarios alberga es "Downtown Toronto" con 17, mientras que los más pequeños, solo albergarían 1 solo vecindario

#### Ahora pintaremos las coordenadas en un mapa para ver la ditribución de los vecindarios

In [41]:
# Para ello recuperaremos las coordenadas de toronto con una librería externa y 
#   pintaremos en un mapa las coordenadas recogidas previamente utilizando los métodos que nos 
#   han enseñado en otras prácticas (como la de NY)
from geopy.geocoders import Nominatim # Librería para recuperar longitudes y latitudes
address = 'Toronto, CA'
geolocation = Nominatim(user_agent = 'CA_explorer')
location = geolocation.geocode(address)
location =np.array(location, dtype=object)

In [42]:
location[1]
log = str(location[1]).split(',')[1].replace(')','')
lat =  str(location[1]).split(',')[0].replace('(','')
print('(' + lat + ',' + log + ' )')

(43.6534817, -79.3839347 )


In [45]:
import folium
mapa_canada = folium.Map(location=[float(lat), float(log)], zoom_start=10)

# añadir marcadores al mapa
for lat, lng, borough, neighborhood in zip(df_clean['Latitude'], df_clean['Longitude'], df_clean['Municipio'], df_clean['Vecindario']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='#f66b4d',
        fill=True,
        fill_color='#cc6d59',
        fill_opacity=0.7,
        parse_html=False).add_to(mapa_canada)  
    
mapa_canada